## This notebook reproduce pore clogging due to too quick SEI layer growth

In [27]:
import matplotlib.pyplot as plt
import numpy as np
import pybamm
import pandas as pd

In [28]:
model = pybamm.lithium_ion.DFN(options={
    "SEI": "interstitial-diffusion limited", 
    "SEI on cracks": "true",
    "particle mechanics": "swelling and cracking",
    "SEI film resistance": "distributed", 
    "SEI porosity change": "true",})
param = pybamm.ParameterValues("OKane2022")
param.update({"Inner SEI lithium interstitial diffusivity [m2.s-1]": 3.5e-16})  
exp = pybamm.Experiment([(
    "Hold at 4.2V until C/100", 
    "Discharge at 1C until 2.5V", 
    "Rest for 1 hours", 
    "Charge at 1C until 4.2V", 
)] *5 )
var_pts = {
    "x_n": 10,  # negative electrode
    "x_s": 10,  # separator 
    "x_p": 10,  # positive electrode
    "r_n": 30,  # negative particle
    "r_p": 20,  # positive particle
}

In [29]:
sim1 = pybamm.Simulation(
    model, 
    experiment=exp,
    parameter_values=param,
    solver=pybamm.CasadiSolver(return_solution_if_failed_early=True),
    var_pts=var_pts,
)
sol1 = sim1.solve(calc_esoh=False)

At t = 534.618, , mxstep steps taken before reaching tout.
At t = 488.629, , mxstep steps taken before reaching tout.
The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
The linesearch algorithm failed with too small a step.
2023-08-28 21:17:10.962 - [ERROR] callbacks.on_experiment_error(224): Simulation error: Maximum number of decreased steps occurred at t=40535.46333172852 (final SolverError: 'Error in Function::call for 'F' [IdasInterface] at .../casadi/core/function.cpp:1401:
Error in Function::call for 'F' [IdasInterface] at .../casadi/core/function.cpp:330:
.../casadi/interfaces/sundials/idas_interface.cpp:596: IDACalcIC returned "IDA_LINESEARCH_FAIL". Consult IDAS documentation.'). For a full solution try reducing dt_max (currently, dt_max=600) and/or reducin

In [30]:
sol1.cycles

In [32]:
Vars = [
    "Negative electrode porosity",
    "Terminal voltage [V]",
]
quick_plot = pybamm.QuickPlot(sol1, Vars,variable_limits='fixed') #     
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=11.259850925479867, step=0.11259850925479867…